I want to parse the histone peptides, marks, m/z, and retention times out of EpiProfile. Example of how that data is put together in EpiProfile code below:

```
function His = init_histone()
%%


His.pep_seq = 'TKQTAR';
His.mod_short = {'unmod';
    'K4me1';
    'K4me2';
    'K4me3';
    'K4ac'};
His.mod_type = {'0,pr;2,pr;';
    '0,pr;2,me1;';
    '0,pr;2,me2;';
    '0,pr;2,me3;';
    '0,pr;2,ac;'};

His.pep_ch = repmat([1 2],length(His.mod_type),1);
%{
His.pep_mz = [816.4574	408.7323
    830.4730	415.7402
    788.4625	394.7349
    802.4781	401.7427
    802.4417	401.7245];
%}
His.pep_mz = calculate_pepmz(His);
His.rt_ref = [18.34
    21.58
    10.82
    10.8
    16.64];
His.display = ones(length(His.mod_type),1);
```


We also want to make this method work for any organism. For that, we'd have the user supply a FASTA with the histone proteins they want to measure, and a list of the histone marks (in the form of `[Single amino acid letter residue][Residue position in organism-specific protein][modification abbreviation][modification duplication]`). We may have to supply our table of histone marks and their respective mass shifts so that the user can correctly set up the histone marks they care about.

In [28]:
import sys
import os
from os import listdir
from os.path import isfile, join
import pandas as pd
import re
import numpy as np
from Bio import SeqIO


##
## scraping out the EpiProfile information into a more useable format
##

def epi_strip(file_in):
    
    #sys.stdout.write("Current file: %s\n" % file_in)
    epiprofile_m = open(os.path.join(proj_dir, file_in))

    text = epiprofile_m.readlines()
    epiprofile_m.close()

    # initializing substring 
    start = 'His.pep_seq ='
    end = 'His.display ='

    # using list comprehension to get string with substring  
    info_start = text.index([i for i in text if start in i][0])
    info_end = text.index([i for i in text if end in i][0])

    info = ''.join(text[info_start:info_end])

    ## clean up the text
    # remove the semicolons and convert more "pythonic"
    info = info.replace(';\n', '\n')

    # comment out the functions
    info = info.replace('His.pep_ch = ', '#His.pep_ch = ')
    info = info.replace('His.pep_mz = calculate_pepmz(His)', '#His.pep_mz = calculate_pepmz(His)')
    info = info.replace('His.pep_mz = calculate_pepmzH', '#His.pep_mz = calculate_pepmzH')

    # replace all 
    info = info.replace('%', '')

    # reformat how the pep_mz values are stored so it's more pythonic
    info = info.replace('\t', '-')
    info = info.replace("{'", "['").replace("'}", "']")
    info = info.replace('\n[\n', '\n\n').replace('\n]\n', '\n\n')

    # restructure the arrays to be more pythonic
    info = info.replace("'\n", "', ").replace("\n    ", ', ')

    # change the pep_mz values
    info = info.replace('{\nHis.pep_mz = [', "\nHis.pep_mz = '").replace(']\n}', "'")

    # remove the 'His.' class(?) from the variables so python isn't confused
    info = info.replace('His.', '')

    # fix the pep_seq, mod_short line that got messed up
    info = info.replace(', mod_short = [', '\nmod_short = [')

    # some additional clean ups...
    info = info.replace('\n\n', '\n')
    info = info.replace('[', '').replace(']', '')
    #print(info)

    test = info.split('\n')
    test = [x for x in test if "#" not in x]

    data_dict = {}
    data_dict['EpiFile'] = [file_in]
    for item in test:
        key = item.split('= ')[0].replace(' ', '').replace("'", "")
        values = item.split('= ')[1:]
        values = [re.sub('\'', '', _) for _ in values]
        #values = values[0].split(',')
        new_data = {key: values}
        if len(new_data[key]) > 0:
            data_dict[key] = values
    temp = pd.DataFrame.from_dict(data_dict)

    return temp


##
## index the first amino acid of each peptide to match the "K4me1" modification, e.g.
##

def find_aa_index(peptide):
    onepep_aa = pd.DataFrame(columns = ['pep_seq', 'protein', 'aa_index'])
    for seq_record in fasta_dict:
        matches = int(fasta_dict[seq_record].seq.find(peptide)) + 1  # add 1 bc python indexing
        if matches > 1:
            new_row = {'pep_seq':peptide, 'protein':seq_record, 'aa_index':matches}
            #print(new_row)
            onepep_aa = onepep_aa.append(new_row, ignore_index=True)
        #print(onepep_aa)
    return onepep_aa


##
## given a combination histone mark, breaks each mark into its component aa, position, and modification
##

def parse_hismarks(his_mark):
    pattern = r'[A-Z]{1}[1-9]+[a-z]+[1-9]*'
    if his_mark is not None:
        matches = re.findall(pattern, his_mark)
    else:
        matches = []
    return matches


##
## map shorthand histone modifications to the peptide aa modification and [m/z] shift
##

def mod_sequence(row):
    
    #print(list(row))

    # parse the row
    if row['aa_index'] > 0:
        peptide_seq = list(row['pep_seq'])  # split the peptide sequence into component amino acids
        peptide_mod_seq = list(row['pep_seq'])  # also make a copy for the [m/z] Peptide Modified Sequence
        index = int(row['aa_index'])  # +1 to get the indexing to work
                  
        mods = parse_hismarks(row['his_marks'])  # get the component marks
        
        # propionylate the first residue
        if peptide_seq[0] != 'K':
            peptide_mod_seq[0] = peptide_mod_seq[0]+'[56.026215]'
            
        for mod in mods:
            #print(mod)
            mod_residue = re.findall(r'[A-Z]+[1-9]+', mod)[0]
            mod_residue = re.findall(r'[1-9]+', mod_residue)[0]
            mod_index = 1+ (int(mod_residue) - index)  # should correspond to peptide_seq index
            mod_aa = re.findall(r'[A-Z]+', mod)[0]
            mod_id = re.findall(r'[a-z]+[1-9]*', mod)[0]
            
            # mark when special ntermK modification
            if (mod_aa == 'K') & (mod_index == 0):
                mod_aa = 'ntermK'

            # add the [m/z] modification shift to the amino acid residue
            if mod_mz_map['mod'].astype(str).str.contains((mod_aa+mod_id)).any():
                mass_shift = mod_mz_map.loc[mod_mz_map['mod'] == (mod_aa+mod_id), 'mass'].iloc[0]
                #print(mass_shift)
                peptide_mod_seq[int(mod_index)] = mass_shift
            else:
                peptide_mod_seq = list('error: modification not in unimod')
                peptide_seq = list('error: modification not in unimod')
                break

            peptide_seq[int(mod_index)] = mod_aa+mod_id

            #print(mod)
            
        # double propionylate previously unmodified N-terminal lysines
        if peptide_seq[0] == 'K':
            peptide_seq[0] = 'ntermK'
            peptide_mod_seq[0] = 'K[112.05243]'
        
        # propionylate any previously unmodified internal K
        peptide_mod_seq = ['K[56.026215]' if aa=='K' else aa for aa in peptide_mod_seq]
        
        row['peptide_seq'] = ''.join(peptide_seq)
        row['Peptide Modified Sequence'] = ''.join(peptide_mod_seq)
        
        return row
    

sys.stdout.write("Imported all packages and functions.\n")

Imported all packages and functions.


In [29]:
proj_dir = os.path.join(os.getcwd(), './epiprofile/')

file_list = [f for f in listdir(proj_dir) if isfile(join(proj_dir, f))]
file_list = [ x for x in file_list if ".m" in x ]

results_df = pd.DataFrame()
for file in file_list:
    results_df = results_df.append(epi_strip(file))

results_df['id'] =  results_df['EpiFile'].map(str).replace('.m', '') + "_" + results_df['pep_seq']  
sys.stdout.write("Finished parsing the EpiProfile code.\n")


# so unfortunately, while we seem to be scraping pretty faithfully, 
# we need to still 'explode' the rows to either modification level or even precursor level

df2 = pd.DataFrame()
for index, row in results_df.iterrows():
    
    col3 = row['mod_short'].replace('\t', '').replace(' ', '').split(',')
    col4 = row['pep_mz'].split(',')
        
    # sometimes EpiProfile doesn't have precursor m/z for histone marks, so add a NaN for those
    if len(col3) > len(col4):
        sys.stderr.write("More histone marks than precursors: %s, %s\n" % 
                         (str(row['EpiFile']), str(row['pep_seq']))  )
        append_nan = np.nan*(len(col3)-len(col4))
        col4 = col4.append(append_nan)
        longer_array = len(col3)
        #print(col4)
    elif len(col3) < len(col4):
        sys.stderr.write("More precursors than histone marks: %s, %s\n" % 
                         (str(row['EpiFile']), str(row['pep_seq']))  )
        append_nan = np.nan*(len(col4)-len(col3))
        col3 = col3.append(append_nan)
        longer_array = len(col4)
        #print(col3)
    else:
        longer_array = len(col3)
    
    col1 = [row["EpiFile"]]*longer_array
    col2 = [row["pep_seq"]]*longer_array
    
    if 'zeros' in ''.join(row['rt_ref']):
        sys.stderr.write("More histone marks than precursors: %s, %s\n" % 
                         (str(row['EpiFile']), str(row["pep_seq"])) )
        col5 = [np.nan]*longer_array
    else:
        col5 = row['rt_ref'].split(',')
    
    data = pd.DataFrame({'EpiFile': col1,
        'pep_seq': col2, 
        'his_marks': col3,
        'precursor_mz': col4,
        'rt_ref': col5})

    df2 = df2.append(data, ignore_index=True)

del results_df

# explode out the precursor m/z column so each row is a precursor
s = df2['precursor_mz'].str.split('-').apply(pd.Series, 1).stack()
s.index = s.index.droplevel(-1)
s.name = 'precursor_mz'
del df2['precursor_mz']

# clean up the dataframe names and collect garbage
df2 = df2.join(s)
results_df = df2
del df2

sys.stdout.write("Finished exploding out the stripped EpiProfile data.\n")

# import Josh's fasta of all human histones
fasta_dict = SeqIO.to_dict(SeqIO.parse(os.path.join(os.getcwd(), "../data/fasta/Histones.fasta"), "fasta"))

# index the first amino acid for all peptides in the Epiprofile code
peptides = results_df['pep_seq'].drop_duplicates()
aa_index_all = pd.DataFrame(columns=['pep_seq', 'protein', 'aa_index'])
for peptide in peptides:
    aa_index_all = aa_index_all.append(find_aa_index(peptide))

results_df = pd.merge(results_df, aa_index_all, on='pep_seq', how='outer')
del aa_index_all
sys.stdout.write("Finished indexing peptide amino acid position relative to proteins in fasta.\n")

## lookup values to make a Peptide Modified Sequence column
mod_mz_map = pd.read_csv('../data/unimod_massshift.csv')
results_df = results_df.apply(mod_sequence, axis=1)
sys.stdout.write("Finished adding modification mass shift(s) to peptide sequences.\n")

results_df.to_csv(os.path.join(os.getcwd(), '../results/epiprofile_strip.csv'), index=False)
results_df.head()

Finished parsing the EpiProfile code.


More histone marks than precursors: H3M_01_3_8.m, TKQTAR
More histone marks than precursors: H3M_01_3_8.m, TKQTAR
More histone marks than precursors: H3M_02_9_17.m, KSTGGKAPR
More histone marks than precursors: H3M_02_9_17.m, KSTGGKAPR
More precursors than histone marks: H3M_03_18_26.m, KQLATKAAR
More histone marks than precursors: H3M_03_18_26.m, KQLATKAAR
More histone marks than precursors: H3M_04v3_27_40.m, KSAPSTGGVKKPHR
More histone marks than precursors: H3M_04v3_27_40.m, KSAPSTGGVKKPHR
More histone marks than precursors: H3M_04_27_40.m, KSAPATGGVKKPHR
More histone marks than precursors: H3M_04_27_40.m, KSAPATGGVKKPHR
More histone marks than precursors: H3M_06_54_63.m, YQKSTELLIR
More histone marks than precursors: H3M_06_54_63.m, YQKSTELLIR
More histone marks than precursors: H3M_07_73_83.m, EIAQDFKTDLR
More histone marks than precursors: H3M_07_73_83.m, EIAQDFKTDLR
More precursors than histone marks: H3M_08_117_128.m, VTIMPKDIQLAR
More histone marks than precursors: H3M_08_117_

Finished exploding out the stripped EpiProfile data.
Finished indexing peptide amino acid position relative to proteins in fasta.
Finished adding modification mass shift(s) to peptide sequences.


,EpiFile,pep_seq,his_marks,rt_ref,precursor_mz,protein,aa_index,peptide_seq,Peptide Modified Sequence
0,H3H_01_3_8.m,TKQTAR,unmod,18.34,816.4574,sp|P68431|H31_HUMAN,4.0,TKQTAR,T[56.026215]K[56.026215]QTAR
1,H3H_01_3_8.m,TKQTAR,unmod,18.34,816.4574,sp|P84243|H33_HUMAN,4.0,TKQTAR,T[56.026215]K[56.026215]QTAR
2,H3H_01_3_8.m,TKQTAR,unmod,18.34,816.4574,sp|Q71DI3|H32_HUMAN,4.0,TKQTAR,T[56.026215]K[56.026215]QTAR
3,H3H_01_3_8.m,TKQTAR,unmod,18.34,816.4574,sp|Q16695|H31T_HUMAN,4.0,TKQTAR,T[56.026215]K[56.026215]QTAR
4,H3H_01_3_8.m,TKQTAR,unmod,18.34,816.4574,sp|Q6NXT2|H3C_HUMAN,4.0,TKQTAR,T[56.026215]K[56.026215]QTAR


In [35]:
# parse out a Skyline-acceptable "Insert Peptides" csv
skyline_peptides = results_df[['Peptide Modified Sequence', 'protein']].drop_duplicates()
skyline_peptides = skyline_peptides[~skyline_peptides['Peptide Modified Sequence'].str.contains("error", na=False)]

skyline_peptides.to_csv(os.path.join(os.getcwd(), '../results/skyline_insert_peptides.csv'), 
                        index=False, header=False)


In [ ]:
# parse out a Skyline-acceptable "Peptide Notes" document grid thing?
